In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [4]:
####Objective of the analysis is to check if there is a drop in the aircon fault after a train aircon has gone through overhaul.

In [25]:
df = pd.read_excel(r"C:\Users\Darius\Downloads\Aircon C830 Analysis\data.xlsx")  #read all the circle line train fault data
tb = pd.read_excel(r"C:\Users\Darius\Downloads\Aircon C830 Analysis\S2 Aircon Overhaul Date.xlsx") #read aircon overhaul date data

In [26]:
df = df[df["Fault Group"] == "Air Condition System"] #only looking for  Work Order (WO) with fault tagged under air con fault
df = df[df["Fleet"] == "C830"]  #only looking for  Work Order (WO) from specific fleet train
df = df[df["Operation Text"] !="Nil"] #only looking for  Work Order (WO) that is completed

In [27]:
#extract the relevant columns
df = df.loc[:,["Record","WO Text","Cause","Operation Text","Train Number"]]

In [14]:
df.head()

,Record,WO Text,Cause,Operation Text,Train Number
0,2016-01-01,All Cars cooling all failed. BLY SS chec,Air Condition System other failure,:-PV8191 A/C2.1 - CB3 tripped. Reset CB3. Wash...,PV19
1,2016-01-01,8031 cooling failed,Compressor Unit failure,":-8031 1.1 - Replaced A/C comp, pressure test,...",PV03
2,2016-01-01,BTN Rover reported Car1 and Car2 warm. S,Central Processing Unit (CPU) failure,:-PV8361 Ventilation fault. Replaced CPU (unab...,PV36
3,2016-01-01,8121/RSC/CAN/FDM/B1 ATS Alarm Manager and TIP ...,Fresh Air Damper (FAD) failure,:-8121 A/C2.1 - FAD sensor adjusted and cleare...,PV12
11,2016-01-04,8061/RSC/ACN,Air Condition System other failure,:-PV8061 A/C2.1 - CB3 tripped. Reset CB3. Func...,PV06


In [30]:
#rename some of the columns to make better sense & align better tb and df

df.rename(columns={"Record": "Fault_Record_Date","WO Text": "Fault Symptoms", "Operation Text": "Fault Findings"}, inplace= True)
tb.rename(columns={"Record": "Overhaul_Record_Date",'S2 Aircon OverhaulActual date':'Record','PV No':'Train Number'}, inplace= True)

#changing the data format 
df['Fault_Record_Date'] = pd.to_datetime(df['Fault_Record_Date'], format = '%Y-%m-%d')
tb['Overhaul_Record_Date'] = pd.to_datetime(tb['Overhaul_Record_Date'], format = '%Y-%m-%d')

#making the train number column str same as df train number column
tb['Train Number'] = tb['Train Number'].str.upper()

# column creation
tb["overhauled_days"] = tb["Overhaul_Record_Date"].max() - tb["Overhaul_Record_Date"] #create a new column to record days that fault happened after train overhauled.
tb["unoverhauled_days"] = tb["Overhaul_Record_Date"]  - pd.to_datetime("2016-01-01", format = '%Y-%m-%d') #create a new column to record days that fault happened before train overhauled.

In [31]:
dfjoined = df.merge(tb, on="Train Number", how='inner') # join df and tb together

In [32]:
dfjoined["fault_after_overhauled"] = "" #create a new column for boolean check and assign 1 to fault happened after overhaul, 0 to fault happened before overhaul.
dfjoined["fault_before_overhauled"] = "" #create a new column for boolean check and assign 1 to fault happened before overhaul, 0 to fault happened after overhaul.

#turn train number from str to int
dfjoined["Train Number"] = dfjoined["Train Number"].str[2:4] 
dfjoined["Train Number"] = dfjoined["Train Number"].astype(int)

In [103]:
#create double for loop to assign before or after to fault happened after/before overhaul.

i=0
x=0

for i in range(41): #checking each PV number on the data 1 by 1
    #print("b")
    for x in range(len(dfjoined)):   #checking on all the rows in dfjoined
        if i == dfjoined["Train Number"][x]:
            #print("x")
            if dfjoined['Overhaul_Record_Date'][x] < dfjoined['Fault_Record_Date'][x]: #check for faults happened after overhaul 
                dfjoined.at[x, "fault_after_overhauled"] = 1                           # assign 1 if statement above is True.
                dfjoined.at[x, "fault_before_overhauled"] = 0
                x =+1
                #print("y")
            elif dfjoined['Overhaul_Record_Date'][x] > dfjoined['Fault_Record_Date'][x]: #check if fault record date happened before overhaul record date
                #print("z")
                dfjoined.at[x, "fault_after_overhauled"] = 0                           # assign before to fault after overhauled cell if condition is True.
                dfjoined.at[x, "fault_before_overhauled"] = 1
                x =+1
                #print("z")
            else:
                dfjoined.at[x, "fault_after_overhauled"] = 0                           
                dfjoined.at[x, "fault_before_overhauled"] = 0
                x =+1                
        else:
             x =+1           
    i =+1
    x = 0

In [234]:
#group data based on train.
dfjoinedpivot = dfjoined.pivot_table(index = ["Train Number", 
                                              "overhauled_days",
                                              "unoverhauled_days"],aggfunc = "sum")  

C:\Users\Darius\anaconda3\lib\site-packages\pandas\core\apply.py:577: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  return f(*args, **kwargs)


In [235]:
#reset index
result = dfjoinedpivot.reset_index()

In [236]:
# create a new column to normalise overhauled days, using fault count.
# higher overhauled_days_per_fault means it takes more days for a fault to happened.
result["overhauled_days_per_fault"] = result["overhauled_days"] / result["fault_after_overhauled"]

# create a new column to normalise unoverhauled days, using fault count. 
# higher unoverhauled_days_per_fault means it takes more days for a fault to happened.
result["unoverhauled_days_per_fault"] = result["unoverhauled_days"] / result["fault_before_overhauled"]

result["overhauled_days_per_fault"].fillna('00:00:00', inplace = True)
result["unoverhauled_days_per_fault"].fillna('00:00:00', inplace = True)

In [237]:
result.head(2)

,Train Number,overhauled_days,unoverhauled_days,Cause,Comments,Fault Findings,Fault Symptoms,fault_after_overhauled,fault_before_overhauled,overhauled_days_per_fault,unoverhauled_days_per_fault
0,1,0 days,2250 days,Air Condition System other failureReturn Air D...,Not yet overhauled. Not yet overhauled. Not ye...,:-PV8011 A/C1.2 - CB5 tripped. Reset CB5/K5. F...,8011/RSC/ACN8013 RAD 1.1 unable to close A/S #...,0,94,0 days,23 days 22:28:05.106382978
1,2,0 days,2250 days,Air Condition System other failureReturn Air D...,Not yet overhauled. Not yet overhauled. Not ye...,":-8021 All units checked and Test Run, cooling...",8021/RSC/ACN8022/RSC/ACN/RDM/A1; 8023/RSC/ACN/...,0,82,0 days,27 days 10:32:11.707317073


In [238]:
#create a new column to check for the result if there is a drop in the aircon fault after a train aircon has gone through overhaul.
result["final_result"] = "" #Qualitative result
result["final_score"] = ""  #Quantitative result

In [239]:
result.head(2)

,Train Number,overhauled_days,unoverhauled_days,Cause,Comments,Fault Findings,Fault Symptoms,fault_after_overhauled,fault_before_overhauled,overhauled_days_per_fault,unoverhauled_days_per_fault,final_result,final_score
0,1,0 days,2250 days,Air Condition System other failureReturn Air D...,Not yet overhauled. Not yet overhauled. Not ye...,:-PV8011 A/C1.2 - CB5 tripped. Reset CB5/K5. F...,8011/RSC/ACN8013 RAD 1.1 unable to close A/S #...,0,94,0 days,23 days 22:28:05.106382978,,
1,2,0 days,2250 days,Air Condition System other failureReturn Air D...,Not yet overhauled. Not yet overhauled. Not ye...,":-8021 All units checked and Test Run, cooling...",8021/RSC/ACN8022/RSC/ACN/RDM/A1; 8023/RSC/ACN/...,0,82,0 days,27 days 10:32:11.707317073,,


In [240]:
#create a foor loop to assign "better", "worst" or "haven't overhauled" between the days from overhauled_days_per_fault and unoverhauled_days_per_fault

i=0
for i in range(len(result)):

    if result.iloc[i].loc["overhauled_days_per_fault"] >= result.iloc[i].loc["unoverhauled_days_per_fault"]:  #check if overhauled_days_per_fault happened is more unoverhauled_days_per_fault
        result["final_result"][i] = "Better"   #higher overhauled_days_per_fault means it takes more days for a fault to happened
        
    else:
        result["final_result"][i] = "Not Better" #else Not Better

C:\Users\Darius\AppData\Local\Temp/ipykernel_8092/808323381.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["final_result"][i] = "Not Better" #else Not Better
C:\Users\Darius\AppData\Local\Temp/ipykernel_8092/808323381.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["final_result"][i] = "Better"   #higher overhauled_days_per_fault means it takes more days for a fault to happened


In [242]:
#To check what is the ratio between overhauled days per fault with unoverhauled days per fault. higher is better.
result["final_score"] = (result['overhauled_days_per_fault']) / (result['unoverhauled_days_per_fault'] )

result = result.sort_values(by = ["final_score"], ascending = False) #sort data by final scores
result = result.loc[result["final_score"] != 0.0]
result = result.loc[:,["Train Number","final_result","final_score"]]
result

,Train Number,overhauled_days,unoverhauled_days,Cause,Comments,Fault Findings,Fault Symptoms,fault_after_overhauled,fault_before_overhauled,overhauled_days_per_fault,unoverhauled_days_per_fault,final_result,final_score
6,7,897 days,1353 days,Dehydrator Filter failureSystem leak (Freon lo...,0,:-PV8073 A/C1.2 - completed servicing. Pressur...,8073 cooling fail.8073/RSC/ACNPV07 ATS Alarm M...,12,62,74 days 18:00:00,21 days 19:44:30.967741935,Better,3.425351
22,23,836 days,1414 days,Supply Air Temperature Sensor (SATS) failureAi...,0,:-PV8232 A/C2.1 - CB3 tripped. Replaced 1x cab...,8232/RSC/ACN8233/RSC/ACN8232/RSC/ACNPV 23 - 82...,7,30,119 days 10:17:08.571428571,47 days 03:12:00,Better,2.533845
15,16,186 days,2064 days,Exterior Smoke Detector failureAir Condition S...,0,:-- Check DDU car 2 Ext. smoke - ok. - Functio...,Train PV16 Intermitent Exterior Smoke De8163/R...,2,50,93 days 00:00:00,41 days 06:43:12,Better,2.252907
35,36,111 days,2139 days,Central Processing Unit (CPU) failureAir Condi...,0,:-PV8361 Ventilation fault. Replaced CPU (unab...,BTN Rover reported Car1 and Car2 warm. S8363/R...,2,58,55 days 12:00:00,36 days 21:06:12.413793103,Better,1.504909
9,10,255 days,1995 days,Supply Air Temperature Sensor (SATS) failureSy...,0,":-8103 vent 1 - Replaced SATs, contactor (full...",8103 - WATER CONDENSATIONATS Alarm Manager & T...,4,46,63 days 18:00:00,43 days 08:52:10.434782608,Better,1.469925
25,26,959 days,1291 days,Dehydrator Filter failureSystem leak (Freon lo...,0,:-PV8261 A/C1.1 - CB1 tripped. ...,Rover reported aircon water condensationPV26PV...,15,27,63 days 22:24:00,47 days 19:33:20,Better,1.337103
19,20,131 days,2119 days,Solenoid Valve failureAir Condition System oth...,0,:-PV8201 A/C1.1 - Pressure out fault. LMV not ...,8201/RSC/ACNPV20PV20 -8201 Cooling failsPV 20 ...,3,62,43 days 16:00:00,34 days 04:15:29.032258064,Better,1.277647
27,28,220 days,2030 days,Solenoid Valve failureAir Condition System oth...,0,:-8283 A/C2.2 - Replaced LMV. Test Run unit co...,8283/RSC/ACN ATS Alarm Manager & TIP showed Tr...,4,44,55 days 00:00:00,46 days 03:16:21.818181818,Better,1.192118
36,37,290 days,1960 days,Evaporator Fan Motor failure.System leak (Freo...,0,:-8371 Replaced Fan Motor impellers (4 nos.). ...,8371 A/C2.2 VENT MOTOR vibrationPV 37 - 8372 C...,5,40,58 days 00:00:00,49 days 00:00:00,Better,1.183673
31,32,74 days,2176 days,Return Air Dampers (RAD) failureFresh Air Damp...,0,:-Check on function test ok. No faults shown.:...,ATS Alarm manager and TIP showed PV32 Car 1 ha...,1,32,74 days 00:00:00,68 days 00:00:00,Better,1.088235


In [218]:
"""
By looking at the mean Work Order count before and after overhaul, we can determine that an overhaul on the aircon does improve the reliability of the aircon.
The improvement is seen on 10/13 (77%) of the trains that went through the aircon overhaul.

This can conclude that train undergo aircon overhaul helps to improve the train aircon reliability. 

There are many questions that we can continue to answer such as which of the components fault has the most impact & least impact from the overhaul.
Also, why did the 3 trains (15,5,22) has a increased mean WO count after overhaul. From my experience, these answers are usually outside the scope of a data analyst. 

So it is probably time to change to an engineer thinking hat!

"""

'\nBy looking at the mean Work Order count before and after overhaul, we can determine that an overhaul on the aircon does improve the reliability of the aircon.\nThe improvement is seen on 10/13 (77%) of the trains that went through the aircon overhaul.\n\nThis can conclude that train undergo aircon overhaul helps to improve the train aircon reliability. \n\nThere are many questions that we can continue to answer such as which of the components fault has the most impact & least impact from the overhaul.\nAlso, why did the 3 trains (15,5,22) has a increased mean WO count after overhaul. From my experience, these answers are usually outside the scope of a data analyst. \n\nSo it is probably time to change to an engineer thinking hat!\n\n'